以SQL語法查找NBA球員資料

# 讀取NBA資料庫

In [1]:
import pandas as pd
import sqlite3

In [2]:
# !pip install ipython-sql

In [3]:
%load_ext sql

In [4]:
%sql sqlite:///nba.db

# 初步檢視資料

In [5]:
%%sql 
SELECT * 
FROM sqlite_master
WHERE type='table';

 * sqlite:///nba.db
Done.


type,name,tbl_name,rootpage,sql
table,players,players,3,"CREATE TABLE players (firstName TEXT, lastName TEXT, temporaryDisplayName TEXT, personId INTEGER PRIMARY KEY REFERENCES career_summaries (personId), teamId INTEGER REFERENCES teams (teamId), jersey INTEGER, pos TEXT, heightFeet INTEGER, heightInches INTEGER, heightMeters REAL, weightPounds REAL, weightKilograms REAL, dateOfBirthUTC TEXT, nbaDebutYear INTEGER, yearsPro INTEGER, collegeName TEXT, lastAffiliation TEXT, country TEXT)"
table,career_summaries,career_summaries,20,"CREATE TABLE career_summaries (personId INTEGER PRIMARY KEY REFERENCES players (personId), tpp REAL, ftp REAL, fgp REAL, ppg REAL, rpg REAL, apg REAL, bpg REAL, mpg REAL, spg REAL, assists INTEGER, blocks INTEGER, steals INTEGER, turnovers INTEGER, offReb INTEGER, defReb INTEGER, totReb INTEGER, fgm INTEGER, fga INTEGER, tpm INTEGER, tpa INTEGER, ftm INTEGER, fta INTEGER, pFouls INTEGER, points INTEGER, gamesPlayed INTEGER, gamesStarted INTEGER, plusMinus INTEGER, min INTEGER, dd2 INTEGER, td3 INTEGER)"
table,teams,teams,2,"CREATE TABLE teams (isNBAFranchise INTEGER, isAllStar INTEGER, city TEXT, altCityName TEXT, fullName TEXT, tricode TEXT, teamId INTEGER PRIMARY KEY REFERENCES players (personId), nickname TEXT, urlName TEXT, teamShortName TEXT, confName TEXT, divName TEXT)"


In [6]:
%%sql
SELECT *
FROM players
LIMIT 2

 * sqlite:///nba.db
Done.


firstName,lastName,temporaryDisplayName,personId,teamId,jersey,pos,heightFeet,heightInches,heightMeters,weightPounds,weightKilograms,dateOfBirthUTC,nbaDebutYear,yearsPro,collegeName,lastAffiliation,country
LeBron,James,"James, LeBron",2544,1610612747,23,F,6,9,2.06,250.0,113.4,1984-12-30,2003,17,St. Vincent-St. Mary HS (OH),St. Vincent-St. Mary HS (OH)/USA,USA
Carmelo,Anthony,"Anthony, Carmelo",2546,1610612757,0,F,6,7,2.01,238.0,108.0,1984-05-29,2003,17,Syracuse,Syracuse/USA,USA


In [7]:
%%sql
SELECT *
FROM career_summaries
LIMIT 2

 * sqlite:///nba.db
Done.


personId,tpp,ftp,fgp,ppg,rpg,apg,bpg,mpg,spg,assists,blocks,steals,turnovers,offReb,defReb,totReb,fgm,fga,tpm,tpa,ftm,fta,pFouls,points,gamesPlayed,gamesStarted,plusMinus,min,dd2,td3
2544,34.5,73.4,50.4,27.0,7.4,7.4,0.8,38.2,1.6,9669,981,2058,4576,1533,8195,9728,12869,25537,1973,5719,7572,10321,2390,35283,1306,1305,7130,49938,507,99
2546,35.1,81.3,44.7,23.2,6.4,2.8,0.5,35.3,1.0,3327,586,1168,2981,1914,5542,7456,9701,21688,1529,4361,6155,7571,3336,27086,1168,1117,1548,41220,172,2


In [8]:
%%sql
SELECT *
FROM teams
LIMIT 2

 * sqlite:///nba.db
Done.


isNBAFranchise,isAllStar,city,altCityName,fullName,tricode,teamId,nickname,urlName,teamShortName,confName,divName
1,0,Atlanta,Atlanta,Atlanta Hawks,ATL,1610612737,Hawks,hawks,Atlanta,East,Southeast
1,0,Boston,Boston,Boston Celtics,BOS,1610612738,Celtics,celtics,Boston,East,Atlantic


# 資料查詢分析

## 最高平均得分前10名球員

In [9]:
%%sql
SELECT RANK() OVER(ORDER BY ppg DESC) AS 排名, temporaryDisplayName AS 球員, ppg AS 平均得分
FROM career_summaries
JOIN players
USING (personId)
ORDER BY ppg DESC
LIMIT 10

 * sqlite:///nba.db
Done.


排名,球員,平均得分
1,"Durant, Kevin",27.1
2,"James, LeBron",27.0
3,"Doncic, Luka",25.6
4,"Harden, James",25.2
5,"Williamson, Zion",25.0
6,"Embiid, Joel",24.7
7,"Lillard, Damian",24.6
8,"Young, Trae",24.0
9,"Davis, Anthony",23.9
10,"Curry, Stephen",23.8


## 累計得分最高的前10名球員

In [10]:
%%sql
SELECT RANK() OVER(ORDER BY points DESC) AS 排名, temporaryDisplayName AS 球員, points AS 累計得分
FROM career_summaries
JOIN players
USING (personId)
ORDER BY points DESC
LIMIT 10

 * sqlite:///nba.db
Done.


排名,球員,累計得分
1,"James, LeBron",35283
2,"Anthony, Carmelo",27086
3,"Durant, Kevin",23491
4,"Harden, James",22022
5,"Westbrook, Russell",21262
6,"Aldridge, LaMarcus",19887
7,"Paul, Chris",19562
8,"Howard, Dwight",18947
9,"Curry, Stephen",17581
10,"DeRozan, DeMar",17216


## 統計各年度進入NBA球員人數

In [19]:
%%sql
SELECT nbaDebutYear AS 出道年份, count(*) AS 球員人數
FROM players
GROUP BY nbaDebutYear

/* 因為這是2020年的現役球員資料，所以不含已退役球員的統計 */

 * sqlite:///nba.db
Done.


出道年份,球員人數
2003,3
2004,3
2005,2
2006,8
2007,6
2008,14
2009,12
2010,12
2011,17
2012,20


## NBA球員所屬國籍統計

In [23]:
%%sql
SELECT country, count(*) AS 球員人數
FROM players
GROUP BY country
ORDER BY 球員人數 DESC
LIMIT 10

 * sqlite:///nba.db
Done.


country,球員人數
USA,368
Canada,17
France,11
Australia,9
Serbia,6
Germany,6
Turkey,4
Spain,4
Croatia,4
Slovenia,3


## 球員來自最多不同國籍的球隊排名

In [28]:
%%sql
SELECT fullName AS 球隊, count(DISTINCT country) AS 不同國籍計數
FROM teams
JOIN players
USING(teamId)
GROUP BY fullName
ORDER BY 不同國籍計數 DESC
LIMIT 10

 * sqlite:///nba.db
Done.


球隊,不同國籍計數
Dallas Mavericks,8
Washington Wizards,7
Chicago Bulls,7
Utah Jazz,6
Toronto Raptors,6
San Antonio Spurs,6
Oklahoma City Thunder,6
New Orleans Pelicans,6
Denver Nuggets,6
Philadelphia 76ers,5


## 找出Lebron James所屬球隊，接著找出球隊所有球員

In [38]:
%%sql
SELECT fullName AS 球隊名稱, temporaryDisplayName AS 球員, pos AS 位置
FROM players
JOIN teams
USING(teamId)
WHERE teamId=(SELECT teamId
FROM players
WHERE temporaryDisplayName='James, LeBron')

 * sqlite:///nba.db
Done.


球隊名稱,球員,位置
Los Angeles Lakers,"James, LeBron",F
Los Angeles Lakers,"Dudley, Jared",F
Los Angeles Lakers,"Gasol, Marc",C
Los Angeles Lakers,"Matthews, Wesley",G
Los Angeles Lakers,"Morris, Markieff",F
Los Angeles Lakers,"Davis, Anthony",F-C
Los Angeles Lakers,"Drummond, Andre",C
Los Angeles Lakers,"Schroder, Dennis",G
Los Angeles Lakers,"Caldwell-Pope, Kentavious",G
Los Angeles Lakers,"Harrell, Montrezl",F-C


## 找出最高及最矮的球員

In [70]:
%%sql
SELECT temporaryDisplayName AS 球員, ROUND(heightMeters*100) AS '身高 (cm)', fullName AS 球隊名稱
FROM players
JOIN teams
USING(teamId)
WHERE heightMeters=(SELECT MAX(heightMeters) FROM players) OR 
      heightMeters=(SELECT MIN(heightMeters) FROM players) 

 * sqlite:///nba.db
Done.


球員,身高 (cm),球隊名稱
"Fall, Tacko",226.0,Boston Celtics
"Harper, Jared",178.0,New York Knicks
"Waters, Tremont",178.0,Boston Celtics
"Howard, Markus",178.0,Denver Nuggets
"Campazzo, Facundo",178.0,Denver Nuggets


## 找出各球隊場均得分最高的球員

In [75]:
%%sql
SELECT fullName AS 球隊, temporaryDisplayName AS 球員, MAX(ppg) AS 場均得分
FROM players
JOIN career_summaries
USING(personId)
JOIN teams
USING(teamId)
GROUP BY teams.teamId

 * sqlite:///nba.db
Done.


球隊,球員,場均得分
Atlanta Hawks,"Young, Trae",24.0
Boston Celtics,"Walker, Kemba",19.8
Cleveland Cavaliers,"Sexton, Collin",19.7
New Orleans Pelicans,"Williamson, Zion",25.0
Chicago Bulls,"LaVine, Zach",18.8
Dallas Mavericks,"Doncic, Luka",25.6
Denver Nuggets,"Jokic, Nikola",18.0
Golden State Warriors,"Curry, Stephen",23.8
Houston Rockets,"Wall, John",19.0
LA Clippers,"George, Paul",20.1


## 不同位置的前3名得分王

In [99]:
%%sql
SELECT * 
FROM (SELECT pos AS 位置, temporaryDisplayName AS 球員, ppg AS 場均得分,
ROW_NUMBER() OVER(Partition BY pos ORDER BY ppg DESC) AS 分位置排名
FROM players
JOIN career_summaries
USING(personId))
WHERE 分位置排名<=3

 * sqlite:///nba.db
Done.


位置,球員,場均得分,分位置排名
C,"Jokic, Nikola",18.0,1
C,"Vucevic, Nikola",16.7,2
C,"Lopez, Brook",16.5,3
C-F,"Embiid, Joel",24.7,1
C-F,"Towns, Karl-Anthony",22.8,2
C-F,"Aldridge, LaMarcus",19.4,3
F,"Durant, Kevin",27.1,1
F,"James, LeBron",27.0,2
F,"Williamson, Zion",25.0,3
F-C,"Davis, Anthony",23.9,1
